In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Randomized Image Sampling for Explanations (RISE)

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
from skimage.transform import resize
from tqdm import tqdm

## Change code below to incorporate your *model* and *input processing*

### Define your model here:

In [ ]:
from keras import backend as K
import tensorflow as tf

In [ ]:
class Model():
    def __init__(self):
      K.set_learning_phase(0)
          

      self.model = tf.keras.models.load_model('path model here')
      self.input_size = (560, 560)
        
    def run_on_batch(self, x):
        return self.model.predict(x)

### Load and preprocess image

In [ ]:
from tensorflow.keras.applications.resnet import preprocess_input#, decode_predictions



In [ ]:
from tensorflow.keras.preprocessing import image
import cv2


def load_img(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, model.input_size)
    x = image.img_to_array(img) / 255
    x = np.expand_dims(img, axis=0)

    x = preprocess_input(x)
    x = x.reshape(1, 560, 560, 1) 
    return img, x


---

## RISE

In [ ]:
def generate_masks(N, s, p1):
    cell_size = np.ceil(np.array(model.input_size) / s)
    up_size = (s + 1) * cell_size

    grid = np.random.rand(N, s, s) < p1
    grid = grid.astype('float32')

    masks = np.empty((N, *model.input_size))

    for i in tqdm(range(N), desc='Generatineg masks'):
        # Random shifts
        x = np.random.randint(0, cell_size[0])
        y = np.random.randint(0, cell_size[1])
        # Linear upsampling and cropping
        masks[i, :, :] = resize(grid[i], up_size, order=1, mode='reflect',
                                anti_aliasing=False)[x:x + model.input_size[0], y:y + model.input_size[1]]
    masks = masks.reshape(-1, *model.input_size, 1)
    return masks

In [ ]:
batch_size = 100

def explain(model, inp, masks):
    preds = []
    # Make sure multiplication is being done for correct axes
    masked = inp * masks
    for i in tqdm(range(0, N, batch_size), desc='Explaining'):
        preds.append(model.run_on_batch(masked[i:min(i+batch_size, N)]))
    preds = np.concatenate(preds)
    sal = preds.T.dot(masks.reshape(N, -1)).reshape(-1, *model.input_size)
    sal = sal / N / p1
    print(sal.shape)
    return sal

---

## Running explanations

In [ ]:
def class_name(idx):
    thisdict = {
  "0": "FM1",
  "1": "FM2",
  "2": "OK",
}
    return thisdict[str(idx)]



In [ ]:
model = Model()


In [ ]:

# N =6000
# s = 3
# p1 = 0.2

masks = generate_masks(N, s, p1)

In [ ]:


for class_idx, i in (2, 42), (2,62), (0, 64), (0, 68), (0, 69), (1, 70), (1, 71):

  img, x = load_img('path test image here)
  
  sal = explain(model, x, masks)

  #class_idx = 2
  #plt.title('Explanation for `{}`'.format(class_name(class_idx)))
  x =("Explanation for {0}  N: {1} s: {2} p1: {3}".format(class_name(class_idx), N, s, p1))
  plt.title(x)                                                                                     
  plt.axis('on')
  plt.imshow(img)
  plt.imshow(sal[class_idx], cmap='jet', alpha=0.5)
  plt.colorbar()
  #plt.show()
  path = 'path to save heatmaps here'+ str(i) + '/' + x 
  plt.savefig(path,format='jpg')
  plt.close()

# prediction = model.run_on_batch(x)
# print(prediction)
# predicted_label = np.argmax(prediction)
# print(predicted_label)
